#            Using Robust Optimization to Assign Events to MIT Women Swimmers

### Model 1: Minimizing total swimmers time 

In [1]:
# Load packages
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays, Distributions
const GRB_ENV = Gurobi.Env(output_flag=0);

In [6]:
# Load data
swimmer_times = CSV.read("swimmer_event_times.csv", DataFrame)
swimmer_stats = CSV.read("swimmer_stats.csv", DataFrame)
individual_event_order = CSV.read("individual_event_order.csv", DataFrame)
relay_event_order = CSV.read("relay_event_order.csv", DataFrame)
opponent_times = CSV.read("opponent_times.csv", DataFrame)

Row,Opponent,Event,Opponent_Time
,String31,String31,Float64
1,Group 1,200 Medley Relay,102.62
2,Group 2,200 Medley Relay,104.83
3,"Bastone, Alexandra",1000 Free,603.44
4,"Wood Prince, Piper",1000 Free,622.34
5,"Cruz-Abrams, Robin",1000 Free,653.79
6,"McDonald, Kaylee",200 Free,111.89
7,"Foster, Payton",200 Free,114.7
8,"Hakonardottir, Kristin",200 Free,116.2
9,"Hamlin, Molly",100 Back,54.94


In [7]:
println(names(opponent_times))

["Opponent", "Event", "Opponent_Time"]


In [8]:
# Create dictionaries
best_times = Dict((row[1], row[2]) => row[3] for row in eachrow(swimmer_times))
mean_times = Dict((row[1], row[2]) => row[3] for row in eachrow(swimmer_stats))
std_dev_times = Dict((row[1], row[2]) => row[4] for row in eachrow(swimmer_stats))
individual_event_order_list = individual_event_order[:, 2]
relay_event_positions = Dict(row[2] => row[3:6] for row in eachrow(relay_event_order))
swimmer_list = unique(swimmer_times[:, 1])
opponent_time_dict = Dict((row[:Event], row[:Opponent]) => row[:Opponent_Time] for row in eachrow(opponent_times))

Dict{Tuple{String31, String31}, Float64} with 41 entries:
  ("50 Free", "Brenner, Mandy")           => 24.15
  ("200 Back", "Brock, Lizzie")           => 126.95
  ("400 Free Relay", "Group 2")           => 216.17
  ("100 Breast", "Augustyn, Gabi")        => 63.68
  ("100 Breast", "Denisenko, Aleksandra") => 65.21
  ("100 Back", "Wieclawek, Blythe")       => 56.6
  ("100 Free", "Brenner, Mandy")          => 52.89
  ("1000 Free", "Cruz-Abrams, Robin")     => 653.79
  ("200 Fly", "Yoon, Grace")              => 125.89
  ("1000 Free", "Wood Prince, Piper")     => 622.34
  ("100 Back", "Hamlin, Molly")           => 54.94
  ("500 Free", "Bastone, Alexandra")      => 291.62
  ("200 IM", "Iannaccone, Stephanie")     => 123.86
  ("100 Back", "Wilhelm, Kiley")          => 58.88
  ("50 Free", "Alas, Isabella")           => 24.02
  ("200 Breast", "Iannaccone, Stephanie") => 136.48
  ("200 Free", "Hakonardottir, Kristin")  => 116.2
  ("50 Free", "Hakonardottir, Kristin")   => 24.75
  ("1000 Free", "

In [4]:
# Extract unique swimmer names from swimmer_times DataFrame
swimmer_list = unique(swimmer_times[:, 1])


31-element Vector{String31}:
 "Adler, Lauren"
 "Augustyn, Kate"
 "Beggs, Christina"
 "Bernard, Sarah"
 "Chen, Olivia"
 "Chun, Sydney"
 "Crane, Jessie"
 "Eppinger, Aria"
 "Feliz, Mary"
 "Griffin, Miri"
 "Kim, Jolie"
 "Kudela, Katie"
 "Levy, Lauren"
 ⋮
 "Shipps, Stella"
 "Simons, Kailey"
 "Smith, Sydney"
 "Swanson, Chloe"
 "Swartwood, Belise"
 "Tang, Natalie"
 "Turvey, Alexandra"
 "Yang, Iris"
 "Yang, Rachel"
 "Yao, Katherine"
 "Zhao, Kathy"
 "Zhu, Emma"

In [9]:
# Parameters
swimmer_count = 31
num_individual_events = 12
num_relay_events = 2 # events 1 and 14
num_relay_positions = 4 # number of positions in each relay
num_groups = 3
num_places = 6 # 6 different places to finish a race

penalty_time = 99999  # A large penalty value to discourage invalid assignments

99999

In [6]:
println("Number of swimmers: ", swimmer_count)
println("Number of events: ", length(individual_event_order_list))
println("Number of relay events: ", num_relay_events)

Number of swimmers: 31
Number of events: 12
Number of relay events: 2


In [7]:
println(individual_event_order_list)
println("Best times dictionary: ", best_times)


String15["1000 Free", "200 Free", "100 Back", "100 Breast", "200 Fly", "50 Free", "100 Free", "200 Back", "200 Breast", "500 Free", "100 Fly", "200 IM"]
Best times dictionary: Dict{Tuple{String31, String15}, Float64}(("Augustyn, Kate", "100 Back") => 56.85, ("Swanson, Chloe", "100 Fly") => 65.54, ("Feliz, Mary", "50 Back") => 29.35, ("Levy, Lauren", "100 Breast") => 71.74, ("Zhao, Kathy", "50 Breast") => 31.65, ("Chun, Sydney", "100 Fly") => 61.63, ("Smith, Sydney", "50 Back") => 27.47, ("Naveen, Annika", "200 Free") => 116.8, ("Bernard, Sarah", "50 Breast") => 29.97, ("Beggs, Christina", "50 Free") => 24.12, ("Lu, Sarah", "100 Free") => 58.63, ("Tang, Natalie", "500 Free") => 315.5, ("Turvey, Alexandra", "200 Free") => 116.97, ("Zhao, Kathy", "50 Free") => 25.96, ("Crane, Jessie", "400 IM") => 277.87, ("Smith, Sydney", "50 Free") => 24.85, ("Simons, Kailey", "50 Breast") => 30.34, ("Augustyn, Kate", "200 Back") => 123.96, ("Eppinger, Aria", "1000 Free") => 668.79, ("Chen, Olivia", "10

In [34]:
missing_count = 0
for s in swimmer_list
    for e in individual_event_order_list
        if !haskey(best_times, (s, e))
            best_times[(s, e)] = penalty_time  # Assign a large penalty time
            missing_count += 1
            println("Added default penalty time for swimmer $s in event $e")
        end
    end
end
println("Total missing keys: ", missing_count)


Added default penalty time for swimmer Adler, Lauren in event 1000 Free
Added default penalty time for swimmer Adler, Lauren in event 200 Free
Added default penalty time for swimmer Adler, Lauren in event 100 Back
Added default penalty time for swimmer Adler, Lauren in event 200 Fly
Added default penalty time for swimmer Adler, Lauren in event 50 Free
Added default penalty time for swimmer Adler, Lauren in event 100 Free
Added default penalty time for swimmer Adler, Lauren in event 200 Back
Added default penalty time for swimmer Adler, Lauren in event 100 Fly
Added default penalty time for swimmer Augustyn, Kate in event 1000 Free
Added default penalty time for swimmer Augustyn, Kate in event 100 Breast
Added default penalty time for swimmer Augustyn, Kate in event 200 Fly
Added default penalty time for swimmer Augustyn, Kate in event 200 Breast
Added default penalty time for swimmer Augustyn, Kate in event 100 Fly
Added default penalty time for swimmer Beggs, Christina in event 100 Br

In [10]:
model_1 = Model(() -> Gurobi.Optimizer(GRB_ENV))
#variables 
@variable(model_1, x[1:swimmer_count, 1:num_individual_events], Bin) #decision variable of if swimmer competes in event 
@variable(model_1, y[1:swimmer_count, 1:num_relay_positions, 1:num_relay_events, 1:num_groups], Bin) #decicsion variable if swimmer is 
#in the position for relay event in a certain group 

@objective(model_1, Min, 
    # Individual events
    sum(get(best_times, (swimmer_list[s], individual_event_order_list[e]), penalty_time) * x[s, e]
        for s in 1:swimmer_count, e in 1:num_individual_events) +
    
    # Relay events
    sum(get(best_times, (swimmer_list[s], relay_event_positions[relay_event_order[r, 2]][i]), penalty_time) * y[s, i, r, g]
        for s in 1:swimmer_count, i in 1:num_relay_positions, r in 1:num_relay_events, g in 1:num_groups))

# Each swimmer swims 3 individual events
@constraint(model_1, [s in 1:swimmer_count], 
    sum(x[s, e] for e in 1:num_individual_events) <= 3)

# Each individual event we have only one swimmer
@constraint(model_1, [e in 1:num_individual_events], 
    sum(x[s, e] for s in 1:swimmer_count) >= 1) 

# 4 persons per relay group
@constraint(model_1, [r in 1:num_relay_events, g in 1:num_groups], 
    sum(y[s, i, r, g] for s in 1:swimmer_count, i in 1:num_relay_positions) == 4)

# Can't swim back-to-back events
for s in 1:swimmer_count
    for e in 1:num_individual_events-1
        @constraint(model_1, x[s, e] + x[s, e+1] <= 1)
    end
end

# Each swimmer can only swim once in first relay
@constraint(model_1, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# Each swimmer can only swim once in last relay
@constraint(model_1, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# If swim in first relay, can't swim first individual event
@constraint(model_1, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) + x[s, 1] <= 1)

# If swim in last relay, can't swim last individual event
@constraint(model_1, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) + x[s, num_individual_events] <= 1)

# Exclude the pairs with missing times
#@constraint(model_1, [s in 1:swimmer_count, e in 1:num_individual_events], 
 #   x[s, e] <= haskey(best_times, (swimmer_list[s], individual_event_order_list[e])))

# Each swimmer can only swim in one position in a group
@constraint(model_1, [r in 1:num_relay_events, g in 1:num_groups, i in 1:num_relay_positions], 
    sum(y[s, i, r, g] for s in 1:swimmer_count) == 1)

#optimizing 
optimize!(model_1)

if termination_status(model_1) == MOI.OPTIMAL
    println("Optimal solution found")
else
    println("No optimal solution found")
end

println("Total swimming time: ", objective_value(model_1))

Optimal solution found
Total swimming time: 2412.7999999999997


In [11]:
#printing out what events the swimmer are swimming and what times 
for s in 1:swimmer_count
    for e in 1:num_individual_events
        if value(x[s, e]) > 0.5
            swimmer = swimmer_list[s]
            event = individual_event_order[e, :event]
            best_time = get(best_times, (swimmer, event), "N/A")
            println("Swimmer $swimmer assigned to event $event (Best time: $best_time)")
        end
    end
end

Swimmer Augustyn, Kate assigned to event 200 Back (Best time: 123.96)
Swimmer Bernard, Sarah assigned to event 200 Breast (Best time: 136.67)
Swimmer Naveen, Annika assigned to event 50 Free (Best time: 23.53)
Swimmer Roberson, Ella assigned to event 1000 Free (Best time: 624.84)
Swimmer Roberson, Ella assigned to event 500 Free (Best time: 303.66)
Swimmer Roberson, Ella assigned to event 200 IM (Best time: 125.25)
Swimmer Simons, Kailey assigned to event 100 Breast (Best time: 60.29)
Swimmer Smith, Sydney assigned to event 200 Free (Best time: 116.24)
Swimmer Smith, Sydney assigned to event 100 Free (Best time: 52.48)
Swimmer Smith, Sydney assigned to event 100 Fly (Best time: 57.21)
Swimmer Swartwood, Belise assigned to event 200 Fly (Best time: 125.71)
Swimmer Yang, Iris assigned to event 100 Back (Best time: 55.8)


In [12]:
#prints out each relay group, the position (SHOULD BE 1-4), the swimmer, and the time 
for r in 1:num_relay_events
    for g in 1:num_groups
        println("Relay $(r), Group $(g):")
        for i in 1:num_relay_positions
            for s in 1:swimmer_count
                if value(y[s, i, r, g]) > 0.5
                    swimmer = swimmer_list[s]
                    event = relay_event_order[r, "swimmer_$(i)_event"]
                    best_time = get(best_times, (swimmer, event), "N/A")
                    println("  Position $(i): $(swimmer) (Best time: $(best_time))")
                end
            end
        end
    end
end

Relay 1, Group 1:
  Position 1: Smith, Sydney (Best time: 27.47)
  Position 2: Simons, Kailey (Best time: 30.34)
  Position 3: Beggs, Christina (Best time: 20.09)
  Position 4: Chen, Olivia (Best time: 24.48)
Relay 1, Group 2:
  Position 1: Augustyn, Kate (Best time: 26.5)
  Position 2: Bernard, Sarah (Best time: 29.97)
  Position 3: Naveen, Annika (Best time: 24.89)
  Position 4: Turvey, Alexandra (Best time: 23.61)
Relay 1, Group 3:
  Position 1: Yang, Iris (Best time: 26.18)
  Position 2: Li, Anna (Best time: 30.55)
  Position 3: Zhao, Kathy (Best time: 25.44)
  Position 4: Poag, Lillian (Best time: 24.25)
Relay 2, Group 1:
  Position 1: Smith, Sydney (Best time: 24.85)
  Position 2: Chen, Olivia (Best time: 24.48)
  Position 3: Beggs, Christina (Best time: 24.12)
  Position 4: Naveen, Annika (Best time: 23.53)
Relay 2, Group 2:
  Position 1: Augustyn, Kate (Best time: 24.88)
  Position 2: Kudela, Katie (Best time: 24.87)
  Position 3: Simons, Kailey (Best time: 24.55)
  Position 4:

In [13]:
#prints out the number of events swimmers are in, both individual and relay
for s in 1:swimmer_count
    individual_events = sum(value(x[s, e]) for e in 1:num_individual_events)
    relay_events = sum(value(y[s, i, r, g]) for i in 1:num_relay_positions, r in 1:num_relay_events, g in 1:num_groups)
    total_events = individual_events + relay_events
    println("Swimmer $(swimmer_list[s]): $(total_events) events ($(individual_events) individual, $(relay_events) relay)")
end

Swimmer Adler, Lauren: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Augustyn, Kate: 3.0 events (1.0 individual, 2.0 relay)
Swimmer Beggs, Christina: 2.0 events (-0.0 individual, 2.0 relay)
Swimmer Bernard, Sarah: 2.0 events (1.0 individual, 1.0 relay)
Swimmer Chen, Olivia: 2.0 events (-0.0 individual, 2.0 relay)
Swimmer Chun, Sydney: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Crane, Jessie: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Eppinger, Aria: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Feliz, Mary: 1.0 events (-0.0 individual, 1.0 relay)
Swimmer Griffin, Miri: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Kim, Jolie: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Kudela, Katie: 1.0 events (-0.0 individual, 1.0 relay)
Swimmer Levy, Lauren: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Li, Anna: 1.0 events (-0.0 individual, 1.0 relay)
Swimmer Lu, Sarah: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Naveen, Annika: 3.0 events (1.0 individual, 

### Model 2: Maximize the points

In [14]:
# Points vectors
p_i = [9, 4, 3, 2, 1, 0]  # Points for individual events
p_r = [11, 4, 2, 0, 0, 0]  # Points for relay events

6-element Vector{Int64}:
 11
  4
  2
  0
  0
  0

In [27]:
# Create model
model_2 = Model(Gurobi.Optimizer)

# Variables
@variable(model_2, x[1:swimmer_count, 1:num_individual_events], Bin)
@variable(model_2, y[1:swimmer_count, 1:num_relay_positions, 1:num_relay_events, 1:num_groups], Bin)
@variable(model_2, z_isk[1:swimmer_count, 1:num_individual_events, 1:num_places], Bin)
@variable(model_2, z_iok[1:num_individual_events, 1:num_places], Bin)
@variable(model_2, z_gk[1:num_groups, 1:num_places], Bin)
@variable(model_2, S_mk[1:3, 1:num_places], Bin)

# Objective function
@objective(model_2, Max, 
    sum(p_i[k] * z_isk[s,i,k] for s in 1:swimmer_count, i in 1:num_individual_events, k in 1:num_places) +
    sum(p_r[k] * z_gk[g,k] for g in 1:num_groups, k in 1:num_places)
)

# Constraints
# Each swimmer swims max 3 individual events
@constraint(model_2, [s in 1:swimmer_count], 
    sum(x[s, i] for i in 1:num_individual_events) <= 3)

# 4 persons per relay group
@constraint(model_2, [r in 1:num_relay_events, g in 1:num_groups], 
    sum(y[s, i, r, g] for s in 1:swimmer_count, i in 1:num_relay_positions) == 4)

# Can't swim back-to-back events
for s in 1:swimmer_count
    for i in 1:num_individual_events-1
        @constraint(model_2, x[s, i] + x[s, i+1] <= 1)
    end
end

# Each swimmer can only swim once in first relay
@constraint(model_2, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# Each swimmer can only swim once in last relay
@constraint(model_2, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# Each swimmer can only swim in one position in a group
@constraint(model_2, [r in 1:num_relay_events, g in 1:num_groups, i in 1:num_relay_positions], 
    sum(y[s, i, r, g] for s in 1:swimmer_count) == 1)

# If swim in first relay, can't swim first individual event
@constraint(model_2, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) + x[s, 1] <= 1)

# If swim in last relay, can't swim last individual event
@constraint(model_2, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) + x[s, num_individual_events] <= 1)

# Placing constraints for individual events
@constraint(model_2, [i in 1:num_individual_events, k in 1:5, s in 1:swimmer_count],
    z_isk[s,i,k] * get(best_times, (swimmer_list[s], individual_event_order_list[i]), penalty_time) <= 
    z_iok[i,k+1] * get(opponent_time_dict, (individual_event_order_list[i], k), penalty_time))

# Placing constraints for relay events
@constraint(model_2, [g in 1:num_groups, k in 1:5],
    z_gk[g,k] * sum(get(best_times, (swimmer_list[s], relay_event_positions[relay_event_order[r, 2]][i]), penalty_time) * y[s,i,r,g] 
        for s in 1:swimmer_count, i in 1:num_relay_positions, r in 1:num_relay_events) 
    <= z_gk[g,k+1] * sum(get(opponent_time_dict, (relay_event_order[r, 2], m), penalty_time) for r in 1:num_relay_events, m in 1:3))

# Only one swimmer/opponent per place per event
@constraint(model_2, [i in 1:num_individual_events, k in 1:num_places],
    sum(z_isk[s,i,k] for s in 1:swimmer_count) + z_iok[i,k] == 1)

# Each swimmer gets only one place per event if they swim
@constraint(model_2, [s in 1:swimmer_count, i in 1:num_individual_events],
    sum(z_isk[s,i,k] for k in 1:num_places) == x[s,i])

# Each relay group gets only one place
@constraint(model_2, [g in 1:num_groups],
    sum(z_gk[g,k] for k in 1:num_places) <= 1)

# Each opponent relay group gets only one place
@constraint(model_2, [m in 1:3],
    sum(S_mk[m,k] for k in 1:num_places) <= 1)

# Solve the model
optimize!(model_2)

if termination_status(model_2) == MOI.OPTIMAL
    println("Optimal solution found")
else
    println("No optimal solution found")
end

println("Total points won: ", objective_value(model_2))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-26
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2836 rows, 3456 columns and 12756 nonzeros
Model fingerprint: 0xdc47360f
Model has 15 quadratic constraints
Variable types: 0 continuous, 3456 integer (3456 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  QMatrix range    [2e+01, 1e+05]
  QLMatrix range   [6e+05, 6e+05]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 1919 rows and 426 columns
Presolve time: 0.10s
Presolved: 4652 rows, 6750 columns, 21717 nonzeros
Variable types: 0 continuous, 6750 integer (6750 binary)
Found heuristic solution: objective 156.0000000

Root relaxation: obj

In [16]:
if termination_status(model_2) == MOI.OPTIMAL
    println("Optimal solution found")
    println("Total points: ", objective_value(model_2))
    
    # Individual events
    for s in 1:swimmer_count
        for i in 1:num_individual_events
            for k in 1:num_places
                if value(z_isk[s,i,k]) > 0.5
                    swimmer = swimmer_list[s]
                    event = individual_event_order_list[i]
                    place = k
                    points = p_i[k]
                    println("Swimmer: $swimmer, Event: $event, Place: $place, Points: $points")
                end
            end
        end
    end
    
    # Relay events
    for g in 1:num_groups
        for k in 1:num_places
            if value(z_gk[g,k]) > 0.5
                println("Relay Group $g, Place: $k, Points: $(p_r[k])")
                for r in 1:num_relay_events
                    for i in 1:num_relay_positions
                        for s in 1:swimmer_count
                            if value(y[s,i,r,g]) > 0.5
                                swimmer = swimmer_list[s]
                                event = relay_event_positions[relay_event_order[r, 2]][i]
                                println("  Swimmer: $swimmer, Position: $i, Event: $event")
                            end
                        end
                    end
                end
            end
        end
    end
else
    println("No optimal solution found")
end

Optimal solution found
Total points: 156.0
Swimmer: Adler, Lauren, Event: 200 Free, Place: 1, Points: 9
Swimmer: Adler, Lauren, Event: 100 Breast, Place: 1, Points: 9
Swimmer: Adler, Lauren, Event: 50 Free, Place: 1, Points: 9
Swimmer: Augustyn, Kate, Event: 100 Back, Place: 1, Points: 9
Swimmer: Augustyn, Kate, Event: 200 Fly, Place: 1, Points: 9
Swimmer: Augustyn, Kate, Event: 100 Free, Place: 1, Points: 9
Swimmer: Beggs, Christina, Event: 200 Free, Place: 3, Points: 3
Swimmer: Beggs, Christina, Event: 200 Back, Place: 1, Points: 9
Swimmer: Beggs, Christina, Event: 500 Free, Place: 1, Points: 9
Swimmer: Bernard, Sarah, Event: 100 Back, Place: 3, Points: 3
Swimmer: Bernard, Sarah, Event: 200 Breast, Place: 1, Points: 9
Swimmer: Bernard, Sarah, Event: 100 Fly, Place: 1, Points: 9
Swimmer: Chen, Olivia, Event: 1000 Free, Place: 1, Points: 9
Swimmer: Chen, Olivia, Event: 100 Breast, Place: 3, Points: 3
Swimmer: Chen, Olivia, Event: 200 IM, Place: 1, Points: 9
Swimmer: Chun, Sydney, Event

In [17]:
# Calculate points for opponent in individual events
opponent_individual_points = 0
for i in 1:num_individual_events
    for k in 1:num_places
        if value(z_iok[i,k]) > 0.5  # Check if the opponent is assigned to place k in event i
            opponent_individual_points += p_i[k]
        end
    end
end

# Calculate points for opponent in relay events
opponent_relay_points = 0
for m in 1:3  # Iterate over opponent relay groups
    for k in 1:num_places
        if value(S_mk[m,k]) > 0.5  # Check if the opponent relay group m is assigned to place k
            opponent_relay_points += p_r[k]
        end
    end
end

# Total opponent points
total_opponent_points = opponent_individual_points + opponent_relay_points

# Print results
println("Opponent points from individual events: ", opponent_individual_points)
println("Opponent points from relay events: ", opponent_relay_points)
println("Total opponent points: ", total_opponent_points)

Opponent points from individual events: 72
Opponent points from relay events: 0
Total opponent points: 72


In [18]:
total_points = sum(p_i[k] * value(z_isk[s,i,k]) for s in 1:swimmer_count, i in 1:num_individual_events, k in 1:num_places) +
               sum(p_i[k] * value(z_iok[i,k]) for i in 1:num_individual_events, k in 1:num_places) +
               sum(p_r[k] * value(z_gk[g,k]) for g in 1:num_groups, k in 1:num_places) +
               sum(p_r[k] * value(S_mk[m,k]) for m in 1:3, k in 1:num_places)
println("Total calculated points: ", total_points)
println("Expected total points: 300")

Total calculated points: 228.0
Expected total points: 300


In [19]:
for i in 1:num_individual_events
    total_placements = sum(value(z_isk[s, i, k]) for s in 1:swimmer_count, k in 1:num_places) + 
                       sum(value(z_iok[i, k]) for k in 1:num_places)
    println("Total placements in event $i: $total_placements")
end

Total placements in event 1: 6.0
Total placements in event 2: 6.0
Total placements in event 3: 6.0
Total placements in event 4: 6.0
Total placements in event 5: 6.0
Total placements in event 6: 6.0
Total placements in event 7: 6.0
Total placements in event 8: 6.0
Total placements in event 9: 6.0
Total placements in event 10: 6.0
Total placements in event 11: 6.0
Total placements in event 12: 6.0


In [20]:
for g in 1:num_groups
    total_relay_placements = sum(value(z_gk[g, k]) for k in 1:num_places)
    println("Total relay placements for group $g: $total_relay_placements")
end
for m in 1:3
    total_opponent_relay_placements = sum(value(S_mk[m, k]) for k in 1:num_places)
    println("Total opponent relay placements for group $m: $total_opponent_relay_placements")
end

Total relay placements for group 1: 1.0
Total relay placements for group 2: 1.0
Total relay placements for group 3: 1.0
Total opponent relay placements for group 1: 0.0
Total opponent relay placements for group 2: 0.0
Total opponent relay placements for group 3: 0.0


In [21]:
for r in 1:num_relay_events
    for m in 1:3
        for k in 1:num_places
            println("Opponent relay group $m in event $r at place $k: ", get(opponent_time_dict, (relay_event_order[r, 2], m), "Missing"))
        end
    end
end

Opponent relay group 1 in event 1 at place 1: Missing
Opponent relay group 1 in event 1 at place 2: Missing
Opponent relay group 1 in event 1 at place 3: Missing
Opponent relay group 1 in event 1 at place 4: Missing
Opponent relay group 1 in event 1 at place 5: Missing
Opponent relay group 1 in event 1 at place 6: Missing
Opponent relay group 2 in event 1 at place 1: Missing
Opponent relay group 2 in event 1 at place 2: Missing
Opponent relay group 2 in event 1 at place 3: Missing
Opponent relay group 2 in event 1 at place 4: Missing
Opponent relay group 2 in event 1 at place 5: Missing
Opponent relay group 2 in event 1 at place 6: Missing
Opponent relay group 3 in event 1 at place 1: Missing
Opponent relay group 3 in event 1 at place 2: Missing
Opponent relay group 3 in event 1 at place 3: Missing
Opponent relay group 3 in event 1 at place 4: Missing
Opponent relay group 3 in event 1 at place 5: Missing
Opponent relay group 3 in event 1 at place 6: Missing
Opponent relay group 1 in ev

In [19]:
opponent_times = CSV.read("opponent_times.csv", DataFrame)
println(first(opponent_times, 5))  # Preview the first 5 rows to verify

5×3 DataFrame
 Row │ Opponent            Event             Opponent_Time 
     │ String31            String31          Float64       
─────┼─────────────────────────────────────────────────────
   1 │ Group 1             200 Medley Relay         102.62
   2 │ Group 2             200 Medley Relay         104.83
   3 │ Bastone, Alexandra  1000 Free                603.44
   4 │ Wood Prince, Piper  1000 Free                622.34
   5 │ Cruz-Abrams, Robin  1000 Free                653.79


In [22]:
println("Relay Event Order:")
for row in eachrow(relay_event_order)
    println(row)
end

Relay Event Order:
DataFrameRow
 Row │ index  event             swimmer_1_event  swimmer_2_event  swimmer_3_event  swimmer_4_event 
     │ Int64  String31          String7          String15         String7          String7         
─────┼─────────────────────────────────────────────────────────────────────────────────────────────
   1 │     1  200 Medley Relay  50 Back          50 Breast        50 Fly           50 Free
DataFrameRow
 Row │ index  event                swimmer_1_event  swimmer_2_event  swimmer_3_event  swimmer_4_event 
     │ Int64  String31             String7          String15         String7          String7         
─────┼────────────────────────────────────────────────────────────────────────────────────────────────
   2 │    14  200 Freestyle Relay  50 Free          50 Free          50 Free          50 Free


In [23]:
println("Relay Group Times Verification:")
for r in 1:num_relay_events
    for m in 1:3
        event_name = relay_event_order[r, :event]  # Event name from relay_event_order
        key = (event_name, m)
        if haskey(opponent_time_dict, key)
            println("Key: $key, Time: ", opponent_time_dict[key])
        else
            println("Missing time for key: $key")
        end
    end
end

Relay Group Times Verification:
Missing time for key: (String31("200 Medley Relay"), 1)
Missing time for key: (String31("200 Medley Relay"), 2)
Missing time for key: (String31("200 Medley Relay"), 3)
Missing time for key: (String31("200 Freestyle Relay"), 1)
Missing time for key: (String31("200 Freestyle Relay"), 2)
Missing time for key: (String31("200 Freestyle Relay"), 3)


In [24]:
println("Individual Event Times Verification:")
for s in swimmer_list
    for e in individual_event_order_list
        key = (s, e)
        if haskey(best_times, key)
            println("Key: $key, Time: ", best_times[key])
        else
            println("Missing time for key: $key")
        end
    end
end

Individual Event Times Verification:
Missing time for key: (String31("Adler, Lauren"), String15("1000 Free"))
Missing time for key: (String31("Adler, Lauren"), String15("200 Free"))
Missing time for key: (String31("Adler, Lauren"), String15("100 Back"))
Key: (String31("Adler, Lauren"), String15("100 Breast")), Time: 68.45
Missing time for key: (String31("Adler, Lauren"), String15("200 Fly"))
Missing time for key: (String31("Adler, Lauren"), String15("50 Free"))
Missing time for key: (String31("Adler, Lauren"), String15("100 Free"))
Missing time for key: (String31("Adler, Lauren"), String15("200 Back"))
Key: (String31("Adler, Lauren"), String15("200 Breast")), Time: 147.13
Key: (String31("Adler, Lauren"), String15("500 Free")), Time: 309.64
Missing time for key: (String31("Adler, Lauren"), String15("100 Fly"))
Key: (String31("Adler, Lauren"), String15("200 IM")), Time: 131.12
Missing time for key: (String31("Augustyn, Kate"), String15("1000 Free"))
Key: (String31("Augustyn, Kate"), Stri

In [23]:
println("All Relay Times in opponent_time_dict:")
for key in keys(opponent_time_dict)
    println("Key: $key, Time: ", opponent_time_dict[key])
end

println("All Individual Times in best_times:")
for key in keys(best_times)
    println("Key: $key, Time: ", best_times[key])
end

All Relay Times in opponent_time_dict:
Key: (String31("50 Free"), String31("Brenner, Mandy")), Time: 24.15
Key: (String31("200 Back"), String31("Brock, Lizzie")), Time: 126.95
Key: (String31("400 Free Relay"), String31("Group 2")), Time: 216.17
Key: (String31("100 Breast"), String31("Augustyn, Gabi")), Time: 63.68
Key: (String31("100 Breast"), String31("Denisenko, Aleksandra")), Time: 65.21
Key: (String31("100 Back"), String31("Wieclawek, Blythe")), Time: 56.6
Key: (String31("100 Free"), String31("Brenner, Mandy")), Time: 52.89
Key: (String31("1000 Free"), String31("Cruz-Abrams, Robin")), Time: 653.79
Key: (String31("200 Fly"), String31("Yoon, Grace")), Time: 125.89
Key: (String31("1000 Free"), String31("Wood Prince, Piper")), Time: 622.34
Key: (String31("100 Back"), String31("Hamlin, Molly")), Time: 54.94
Key: (String31("500 Free"), String31("Bastone, Alexandra")), Time: 291.62
Key: (String31("200 IM"), String31("Iannaccone, Stephanie")), Time: 123.86
Key: (String31("100 Back"), Strin

In [25]:
# Calculate points from individual events
total_individual_points = sum(
    p_i[k] * value(z_isk[s, i, k]) 
    for s in 1:swimmer_count, i in 1:num_individual_events, k in 1:num_places
)

println("Total points from individual events: ", total_individual_points)

# Calculate points from relay events
total_relay_points = sum(
    p_r[k] * value(z_gk[g, k])
    for g in 1:num_groups, k in 1:num_places
)

println("Total points from relay events: ", total_relay_points)

# Total points
println("Total team points: ", total_individual_points + total_relay_points)

Total points from individual events: 156.0
Total points from relay events: 0.0
Total team points: 156.0


In [26]:
println("Relay group placements:")
for g in 1:num_groups
    for k in 1:num_places
        if value(z_gk[g, k]) > 0.5
            println("Group $g placed at position $k with points $(p_r[k])")
        end
    end
end

Relay group placements:
Group 1 placed at position 6 with points 0
Group 2 placed at position 6 with points 0
Group 3 placed at position 6 with points 0


In [26]:
println("Relay positions in best_times: ", keys(best_times))
println("Relay event positions: ", relay_event_positions)

Relay positions in best_times: Tuple{String31, String15}[("Augustyn, Kate", "100 Back"), ("Swanson, Chloe", "100 Fly"), ("Feliz, Mary", "50 Back"), ("Levy, Lauren", "100 Breast"), ("Zhao, Kathy", "50 Breast"), ("Chun, Sydney", "100 Fly"), ("Smith, Sydney", "50 Back"), ("Naveen, Annika", "200 Free"), ("Bernard, Sarah", "50 Breast"), ("Beggs, Christina", "50 Free"), ("Lu, Sarah", "100 Free"), ("Tang, Natalie", "500 Free"), ("Turvey, Alexandra", "200 Free"), ("Zhao, Kathy", "50 Free"), ("Crane, Jessie", "400 IM"), ("Smith, Sydney", "50 Free"), ("Simons, Kailey", "50 Breast"), ("Augustyn, Kate", "200 Back"), ("Eppinger, Aria", "1000 Free"), ("Chen, Olivia", "100 Free"), ("Chun, Sydney", "200 Fly"), ("Poag, Lillian", "100 Free"), ("Yao, Katherine", "500 Free"), ("Tang, Natalie", "200 Fly"), ("Augustyn, Kate", "50 Back"), ("Pierce, Camila", "1650 Free"), ("Shipps, Stella", "100 Fly"), ("Crane, Jessie", "500 Free"), ("Simons, Kailey", "50 Fly"), ("Shipps, Stella", "500 Free"), ("Roberson, Ell

In [27]:
println("Opponent relay placements in opponent_time_dict: ", keys(opponent_time_dict))

Opponent relay placements in opponent_time_dict: Tuple{String31, String31}[("50 Free", "Brenner, Mandy"), ("200 Back", "Brock, Lizzie"), ("400 Free Relay", "Group 2"), ("100 Breast", "Augustyn, Gabi"), ("100 Breast", "Denisenko, Aleksandra"), ("100 Back", "Wieclawek, Blythe"), ("100 Free", "Brenner, Mandy"), ("1000 Free", "Cruz-Abrams, Robin"), ("200 Fly", "Yoon, Grace"), ("1000 Free", "Wood Prince, Piper"), ("100 Back", "Hamlin, Molly"), ("500 Free", "Bastone, Alexandra"), ("200 IM", "Iannaccone, Stephanie"), ("100 Back", "Wilhelm, Kiley"), ("50 Free", "Alas, Isabella"), ("200 Breast", "Iannaccone, Stephanie"), ("200 Free", "Hakonardottir, Kristin"), ("50 Free", "Hakonardottir, Kristin"), ("1000 Free", "Bastone, Alexandra"), ("200 Free", "Foster, Payton"), ("200 Back", "Hamlin, Molly"), ("200 Breast", "Yoon, Grace"), ("100 Breast", "Iannaccone, Stephanie"), ("500 Free", "McDonald, Kaylee"), ("200 Fly", "Lu, Sydney"), ("100 Fly", "Lu, Sydney"), ("200 IM", "Augustyn, Gabi"), ("400 Free 

In [199]:
# 2nd model: max time
using JuMP, Gurobi

# Create the model
model = Model(Gurobi.Optimizer)

# Define parameters
swimmer_count = 31
num_individual_events = 12
num_relay_events = 2
num_relay_positions = 4
num_groups = 3
num_places = 6

# Define points vectors (example values, adjust as needed)
p = [9, 4, 3, 2, 1,0]  # Points for individual events
P = [11, 4, 2, 0, 0, 0]  # Points for relay events

# Define variables
@variable(model, x[1:swimmer_count, 1:num_individual_events], Bin)
@variable(model, y[1:swimmer_count, 1:num_relay_positions, 1:num_relay_events, 1:num_groups], Bin)
@variable(model, z_isk[1:swimmer_count, 1:num_individual_events, 1:num_places], Bin)
@variable(model, z_iok[1:num_individual_events, 1:num_places], Bin)
@variable(model, z_gk[1:num_groups, 1:num_places], Bin)
@variable(model, S_mk[1:3, 1:num_places], Bin)

# Objective function
@objective(model, Max, 
    sum(p[k] * z_isk[s,i,k] for s in 1:swimmer_count, i in 1:num_individual_events, k in 1:num_places) +
    sum(P[k] * z_gk[g,k] for g in 1:num_groups, k in 1:num_places)
)

# Constraints
# Each swimmer swims max 3 individual events
@constraint(model, [s in 1:swimmer_count], 
    sum(x[s,i] for i in 1:num_individual_events) <= 3)

# 4 persons per relay group
@constraint(model, [g in 1:num_groups], 
    sum(y[s,r,1,g] for s in 1:swimmer_count, r in 1:4) == 4)
@constraint(model, [g in 1:num_groups], 
    sum(y[s,r,2,g] for s in 1:swimmer_count, r in 5:8) == 4)

# Can't swim back-to-back events
@constraint(model, [s in 1:swimmer_count, i in 1:num_individual_events-1],
    x[s,i] + x[s,i+1] <= 1)

# Each swimmer can only swim once in first relay
@constraint(model, [s in 1:swimmer_count],
    sum(y[s,r,1,g] for r in 1:4, g in 1:num_groups) <= 1)

# Each swimmer can only swim once in last relay
@constraint(model, [s in 1:swimmer_count],
    sum(y[s,r,2,g] for r in 5:8, g in 1:num_groups) <= 1)

# If swim in first relay, can't swim first individual event
@constraint(model, [s in 1:swimmer_count],
    sum(y[s,r,1,g] for r in 1:4, g in 1:num_groups) + x[s,1] <= 1)

# If swim in last relay, can't swim last individual event
@constraint(model, [s in 1:swimmer_count],
    sum(y[s,r,2,g] for r in 5:8, g in 1:num_groups) + x[s,num_individual_events] <= 1)

# Only one swimmer/opponent per place per event
@constraint(model, [i in 1:num_individual_events, k in 1:num_places],
    sum(z_isk[s,i,k] for s in 1:swimmer_count) + z_iok[i,k] == 1)

# Each swimmer gets only one place per event if they swim
@constraint(model, [s in 1:swimmer_count, i in 1:num_individual_events],
    sum(z_isk[s,i,k] for k in 1:num_places) == x[s,i])

# Each relay group gets only one place
@constraint(model, [g in 1:num_groups],
    sum(z_gk[g,k] for k in 1:num_places) <= 1)

# Each opponent relay group gets only one place
@constraint(model, [m in 1:3],
    sum(S_mk[m,k] for k in 1:num_places) <= 1)

# Solve the model
optimize!(model)

# Print results
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found")
    println("Objective value: ", objective_value(model))
    # Add code to print detailed results
else
    println("No optimal solution found")
end


### Model 3: Maximize the points under uncertainty in times